\pagebreak

## Weather Analysis.ipynb (Weather Analysis File)

In [ ]:
import numpy as np
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import box
import matplotlib.pyplot as plt
import seaborn as sns

\pagebreak

### Reading and merging data

In [ ]:
#add traffic incidents
incidents_df = pd.read_csv('..\..\CSV_files\Traffic_Incidents.csv') #get data from CSV file
incidents_df['data']='TrafficIncidents' #create "data" column to record which dataframe this data came from for later
#rename columns so they match other dataframes
incidents_df=incidents_df.rename(columns={'Latitude':'latitude','Longitude':'longitude'})
#incidents_df.head() #<-- used for visual QC

In [ ]:
#split Start_DT into Day/Month/Year (because only interested in 2018)
incidents_df = pd.DataFrame(incidents_df)
incidents_df['MonthDayYear']=incidents_df['START_DT'].str[:10] #re-arrage columns values, and rename
#incidents_df.head() #<-- used for visual QC

In [ ]:
incidents_df = incidents_df.groupby('MonthDayYear')['Count','longitude','latitude','MonthDayYear'].sum()
incidents_df['MonthDayYear']= incidents_df.index
#incidents_df #<-- for visual QC (dont worry, there will be a python warning when this code executes, that is because
#                   the index has the same title as another column in the dataframe, but this column will soon 
#                    me altered in later steps)

In [ ]:
#Seperating the Year,Day and Month from a combined column in the DataFrame. This later allows us to plot more interesting plots
incidents_df = pd.DataFrame(incidents_df)
incidents_df['Year']=incidents_df['MonthDayYear'].str[6:10]
incidents_df = incidents_df.loc[incidents_df['Year'] == '2018'] #only interested in 2018
incidents_df['Day']=incidents_df['MonthDayYear'].str[3:5]
incidents_df['Month']=incidents_df['MonthDayYear'].str[:2]
#incidents_df #<-- used to visualy QC DataFrame

In [ ]:
#need to change datatypes of Year, Month and Day from Object to Integer for the merge of dataframes
incidents_df['Year']= incidents_df['Year'].astype(str).astype(int)
incidents_df['Month']= incidents_df['Month'].astype(str).astype(int)
incidents_df['Day']= incidents_df['Day'].astype(str).astype(int)

In [ ]:
#Lets get the Weather Daily Data Now
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# returns a DataFrame with weather data from "climate.weather.gc.ca"
def download_weather_data(station, year, month=1, daily=True):

    # url to retrieve hourly data
    url_template_hourly = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station}&Year={year}&Month={month}&Day=14&timeframe=1&submit=Download+Data"

    # url to retrieve daily data
    url_template_daily = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station}&Year={year}&Month={month}&Day=14&timeframe=2&submit=Download+Data"

    # daily data by default
    if(daily == True):
        url = url_template_daily.format(station=station, year=year, month=month)
    
    # hourly data when (daily == False)
    else:
        url = url_template_hourly.format(station=station, year=year, month=month)

    # read data into dataframe, use headers and set Date/Time column as index
    weather_data = pd.read_csv(url, index_col='Date/Time', parse_dates=True)

    # replace the degree symbol in the column names
    weather_data.columns = [col.replace('\xb0', '') for col in weather_data.columns]

    return weather_data

In [ ]:
df = download_weather_data(50430, 2018) #<-- get hourly weather

In [ ]:
#rename some columns to make them match other dataframes
df = df.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Daily_Weather = df[['latitude','longitude','Station Name','Year','Month','Day','Mean Temp (C)']]
Daily_Weather['data']='Weather'
#Daily_Weather <-- visual QC

In [ ]:
Hourly_Weather = download_weather_data(50430, 2018, daily=False) # <-- get hourly Data (visibility)
#rename some columns to make them match other dataframes, also add column to record where this data came from for after merge
Hourly_Weather = Hourly_Weather.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Hourly_Weather['data']='Weather'
#Hourly_Weather.head() #<-- visual QC

In [ ]:
#groupby.mean() on "Day" column, to get the average visibulity for each day (daily data was requested for thsi report)
Hourly_Weather = Hourly_Weather.groupby('Day')['longitude','latitude','Visibility (km)','Year','Month','Day'].mean()
#Hourly_Weather.head() #<-- will get a warning because index has same name as a column, but it is okay for now.

In [ ]:
#continue with above strategy, pulling in all weather data for each month
Hourly_Weather2 = download_weather_data(50430, 2018, month = 2, daily=False)
Hourly_Weather2 = Hourly_Weather2.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Hourly_Weather2['data']='Weather'
Hourly_Weather2 = Hourly_Weather2.groupby('Day')['longitude','latitude','Visibility (km)','Year','Month','Day'].mean()

Hourly_Weather3 = download_weather_data(50430, 2018, month = 3, daily=False)
Hourly_Weather3 = Hourly_Weather3.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Hourly_Weather3['data']='Weather'
Hourly_Weather3 = Hourly_Weather3.groupby('Day')['longitude','latitude','Visibility (km)','Year','Month','Day'].mean()

Hourly_Weather4 = download_weather_data(50430, 2018, month = 4, daily=False)
Hourly_Weather4 = Hourly_Weather4.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Hourly_Weather4['data']='Weather'
Hourly_Weather4 = Hourly_Weather4.groupby('Day')['longitude','latitude','Visibility (km)','Year','Month','Day'].mean()

Hourly_Weather5 = download_weather_data(50430, 2018, month = 5, daily=False)
Hourly_Weather5 = Hourly_Weather5.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Hourly_Weather5['data']='Weather'
Hourly_Weather5 = Hourly_Weather5.groupby('Day')['longitude','latitude','Visibility (km)','Year','Month','Day'].mean()

Hourly_Weather6 = download_weather_data(50430, 2018, month = 6, daily=False)
Hourly_Weather6 = Hourly_Weather6.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Hourly_Weather6['data']='Weather'
Hourly_Weather6 = Hourly_Weather6.groupby('Day')['longitude','latitude','Visibility (km)','Year','Month','Day'].mean()

Hourly_Weather7 = download_weather_data(50430, 2018, month = 7, daily=False)
Hourly_Weather7 = Hourly_Weather7.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Hourly_Weather7['data']='Weather'
Hourly_Weather7 = Hourly_Weather7.groupby('Day')['longitude','latitude','Visibility (km)','Year','Month','Day'].mean()

Hourly_Weather8 = download_weather_data(50430, 2018, month = 8, daily=False)
Hourly_Weather8 = Hourly_Weather8.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Hourly_Weather8['data']='Weather'
Hourly_Weather8 = Hourly_Weather8.groupby('Day')['longitude','latitude','Visibility (km)','Year','Month','Day'].mean()

Hourly_Weather9 = download_weather_data(50430, 2018, month = 9, daily=False)
Hourly_Weather9 = Hourly_Weather9.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Hourly_Weather9['data']='Weather'
Hourly_Weather9 = Hourly_Weather9.groupby('Day')['longitude','latitude','Visibility (km)','Year','Month','Day'].mean()

Hourly_Weather10 = download_weather_data(50430, 2018, month = 10, daily=False)
Hourly_Weather10= Hourly_Weather10.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Hourly_Weather10['data']='Weather'
Hourly_Weather10 = Hourly_Weather10.groupby('Day')['longitude','latitude','Visibility (km)','Year','Month','Day'].mean()

Hourly_Weather11 = download_weather_data(50430, 2018, month = 11, daily=False)
Hourly_Weather11 = Hourly_Weather11.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Hourly_Weather11['data']='Weather'
Hourly_Weather11 = Hourly_Weather11.groupby('Day')['longitude','latitude','Visibility (km)','Year','Month','Day'].mean()

Hourly_Weather12 = download_weather_data(50430, 2018, month = 12, daily=False)
Hourly_Weather12 = Hourly_Weather12.rename(columns={"Longitude (x)":"longitude" , "Latitude (y)":"latitude"})
Hourly_Weather12['data']='Weather'
Hourly_Weather12 = Hourly_Weather12.groupby('Day')['longitude','latitude','Visibility (km)','Year','Month','Day'].mean()

#combined all data
hourly_Weather_Total = pd.concat([Hourly_Weather,Hourly_Weather2,Hourly_Weather3,Hourly_Weather4,Hourly_Weather5,\
                                  Hourly_Weather6,Hourly_Weather7, Hourly_Weather8, Hourly_Weather9, Hourly_Weather10,\
                                 Hourly_Weather11, Hourly_Weather12])
#hourly_Weather_Total

In [ ]:
#JOIN HOURLY WEATHER TO DAILY WEATHER
Daily_Weather.index.name=None
hourly_Weather_Total.index.name=None
total_weather = pd.merge(left=hourly_Weather_Total, right=Daily_Weather, how='outer', left_on=['Year','Month','Day'], right_on=['Year','Month','Day'])
#total_weather.head() #<-- QC check

\pagebreak

### Table of general statistics of temperature and visibility (Table)

In [ ]:
analysis = total_weather[['Visibility (km)','Mean Temp (C)']] # only keep columns we are interested in
analysis.describe() # <-- unhide to see table of temperature and visibility statistics in 2018

In [ ]:
#combine Total Weather with collition Data
Colition_weather_df = pd.merge(left=total_weather, right=incidents_df, how='left', left_on=['Year','Month','Day'], right_on=['Year','Month','Day'])
Colition_weather_df=Colition_weather_df.reset_index()
#Colition_weather_df #<-- visual QC
#Colition_weather_df.to_csv("C:/Users/adamd/Desktop/WeatherIncidents.csv") <- hard copy QC

\pagebreak

### Number of days at each temperature (Histogram)

In [ ]:
bins = [-30,-25,-20,-15,-10,-5,0,5,10,15,20,25,30]

fig,ax = plt.subplots(figsize=(15,7))
sns.distplot(Daily_Weather['Mean Temp (C)'],kde=False , bins = bins, hist_kws={"rwidth":0.8,'edgecolor':'black', 'alpha':1.0} )
plt.ylabel("Days at Temperature",fontsize=20)
plt.xlabel("Temperature (C)",fontsize=20)
plt.title("Number of days at each Temperature",fontsize=24)

\pagebreak

### Daily incidents vs. binned mean temperature (C) (PointPlot)

In [ ]:
# Your solution goes here
bins = [-30,-25,-20,-15,-10,-5,0,5,10,15,20,25,30]
Colition_weather_df['Temp Binned'] = pd.cut(Colition_weather_df['Mean Temp (C)'], bins=bins)

fig,ax = plt.subplots(figsize=(15,7))
g = sns.pointplot(x="Temp Binned", y="Count", data=Colition_weather_df , ci = None)
g.set(ylim=(0, 50))

plt.ylabel("Daily Incidents",fontsize=20)
plt.xlabel("Binned Mean Temperature (C)",fontsize=20)
plt.title("Daily Incidents vs. Binned Mean Temperature (C)",fontsize=24)
#Even though it looks like more colisions might happen around the temperatures of -10 and +10, it is only because there are
#more days at those temperatures (see above). However % wise, there are very few days netween -20 and -20 C, yes we still have
#on average 13 to 16 incidents

\pagebreak

### Mean temperature (C) per day (PointPlot)

In [ ]:
fig,ax = plt.subplots(figsize=(15,7))
sns.set(style="darkgrid")
g = sns.pointplot(x="index", y="Mean Temp (C)", data=Colition_weather_df, ci=None)
ax.xaxis.set_major_formatter(plt.NullFormatter())

plt.ylabel("Mean Temperature (C)",fontsize=20)
plt.xlabel("Jan        Feb        Mar        Apr        May        Jun       Jul       Aug       Sep       Oct       Nov       Dec",fontsize=20)
plt.title("Mean Temperature (C) per Day",fontsize=24)

\pagebreak

### Mean temperature (C) per month (PointPlot)

In [ ]:
#PLotting some daily Weather Conditions (Temp and Visibility)
fig,ax = plt.subplots(figsize=(15,7))
sns.set(style="darkgrid")
g = sns.pointplot(x="Month", y="Mean Temp (C)", data=Colition_weather_df, ci=None)

plt.ylabel("Mean Temperature (C)",fontsize=20)
plt.xlabel("Month",fontsize=20)
plt.title("Mean Temperature (C) per Month",fontsize=24)

\pagebreak

### Daily incidents per month (PointPlot)

In [ ]:
#PLotting some daily Weather Conditions (Temp and Visibility)
fig,ax = plt.subplots(figsize=(15,7))
sns.set(style="darkgrid")
g = sns.pointplot(x="Month", y="Count", data=Colition_weather_df, ci=None)

plt.ylabel("Daily Incidents",fontsize=20)
plt.xlabel("Month",fontsize=20)
plt.title("Daily Incidents per Month",fontsize=24)

\pagebreak

### Average visibility (km)  per day (PointPlot)

In [ ]:
fig,ax = plt.subplots(figsize=(15,7))
sns.set(style="darkgrid")
g = sns.pointplot(x="index", y="Visibility (km)", data=Colition_weather_df, ci=None)

ax.xaxis.set_major_formatter(plt.NullFormatter())

plt.ylabel("Visibility (km)",fontsize=20)
plt.xlabel("Jan        Feb        Mar        Apr        May        Jun       Jul       Aug       Sep       Oct       Nov       Dec",fontsize=20)
plt.title("Visibility (km)  per Day",fontsize=24)

\pagebreak

### Average visibility (km) per month (PointPlot)

In [ ]:
fig,ax = plt.subplots(figsize=(15,7))
sns.set(style="darkgrid")
g = sns.pointplot(x="Month", y="Visibility (km)", data=Colition_weather_df, ci=None )

plt.ylabel("Average Vsisibility (km)",fontsize=20)
plt.xlabel("Month",fontsize=20)
plt.title("Average Visibility (km) per Month",fontsize=24)

In [ ]:
#Needed to calculate min and max of visibility data, this will be used to help determine bin sizes for following figures
maximum = Colition_weather_df['Visibility (km)'].max()
minimum = Colition_weather_df['Visibility (km)'].min()
#print(maximum, minimum) #<-- visual QC

\pagebreak

### Daily incidents vs visibility (km) (PointPlot)

In [ ]:
#create plot of Daily Incidents vs. Average Visibility
Colition_weather_df= Colition_weather_df.sort_values(by=['Visibility (km)'])

labels = ['0 to 2km','2 to 4km','4 to 6km','6 to 8km','8 to 10km','10 to 15km','15 to 20km','20 to 25km','25 to 30km','30 to 35km','35 to 40km','40 to 45km','45 to 50km','50 to 55km']
bins = [0,2,4,6,8,10,15,20,25,30,35,40,45,50,55]

Colition_weather_df['visibility_binned']= pd.cut(Colition_weather_df['Visibility (km)'], bins=bins, labels=labels)

fig,ax = plt.subplots(figsize=(15,7))
sns.set(style="darkgrid")
g = sns.pointplot(x="visibility_binned", y="Count", data=Colition_weather_df, ci=None).set_title('Road Incidents vs. Road Visiblity (km)')

plt.ylabel("Daily Incidents",fontsize=20)
plt.xlabel("Average Visibility (km)",fontsize=20)
plt.title("Daily Incidents vs Visibility (km)",fontsize=24)

In [ ]:
#add traffic incidents (but first need to make columns match the weather data column formats)
incidents_df = pd.read_csv('..\..\CSV_files\Traffic_Incidents.csv')
incidents_df.head(10)
incidents_df['data']='TrafficIncidents'
#rename columns so they match other dataframes
incidents_df=incidents_df.rename(columns={'Latitude':'latitude','Longitude':'longitude'})
incidents_df=pd.DataFrame(incidents_df)
incidents_df['Time'] = incidents_df['START_DT'].str[10:]
#incidents_df.head() #<-- visual QC

In [ ]:
#split the Time from the hours in the Incident Data. We need to make the columns match the weather data so we can combined them
import numpy as np
incidents_df['Time'].str[10:]
incidents_df['Time'].str[:3]
incidents_df['night/day'] = incidents_df['Time'].str[10:]
incidents_df['Hour'] = incidents_df['Time'].str[:3]           
incidents_df['Hour'] = incidents_df['Hour'].astype(int)
#incidents_df.head()#<-- visual QC

In [ ]:
#need to change PM/AM time into 24hour clock time, so that we can plot them easier on a graph on the X-axis
#create a column for the addition to the current time: 0 for AM's, and +12 for PM times:
incidents_df["temp"] = incidents_df["night/day"].map(lambda x: '0' if "AM" in x else '12' if "PM" in x else "")
incidents_df['temp'] = incidents_df['temp'].astype(int) #change type to int, so we can add columns together
#incidents_df.head() #<-- visual QC

In [ ]:
incidents_df['24HourClock']=incidents_df['Hour']+incidents_df['temp'] #add columns to get 24hour clock time
#incidents_df.head()#<-- visual QC

\pagebreak

### Total incidents in 2018, for each hour of the day (Histogram)

In [ ]:
#Finaly, plot total incidents vs. time of day
bins = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]

fig,ax = plt.subplots(figsize=(15,7))
sns.distplot(incidents_df['24HourClock'],kde=False , bins = bins, hist_kws={"rwidth":0.8,'edgecolor':'black', 'alpha':1.0} )
plt.ylabel("Total Incidents",fontsize=20)
plt.xlabel("Time of Day (24hour clock)",fontsize=20)
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24])
plt.title("Total Incidents in 2018, For Each Hour of the Day",fontsize=24)
